Q1. chapter3에서 살펴본 기존 CBOW모델의 문제점 2가지는 무엇인가요?

- 1. 어휘 수가 많아지면 입력층의 원핫 표현의 벡터 크기가 커지고, 이에 따라 많은 계산이 필요하게 된다.
- 2. 어휘가 많아지게 되면 은닉층과 가중치 행력의 곱 & Softmax 계층의 계산이 증가한다.

Q2. Embedding 계층이란 무엇인가요?
- 답 : 가중치 매개변수로부터 '단어 ID에 해당하는 행(벡터)'을 추출하는 계층

Q3. Embedding 계층을 Class형태로 구현해주세요.

In [1]:
# 아래에 코드를 작성해주세요.
class Embedding:
  def __init__(self, W):
    self.params = [W]
    self.grads = [np.zeros_like(W)]
    self.idx = None

  def forward(self, idx):
    W, = self.params
    self.idx = idx
    out = W[idx]
    return out

  def backward(self, dout):
    dW, = self.grads
    dW[...] = 0
    if GPU:
      import cupyx
      cupyx.scatter_add(dW, self.idx, dout)
    else:
      np.add.at(dW, self.idx, dout)



Q4. Negative Sampling이란 무엇인가요?

- 답 :  word2vec을 만들 때 현재 문장에 없는 단어를 전체 데이터 셋에서 추출하는 방법

Q5. 다중 분류와 이중 분류에서 각각의 활성화 함수와 손실 함수는 무엇이고, 왜 그런 함수를 사용하는지 적어주세요.

- 답 : 이진 분류와 다중 분류 모두 손실 함수로 '교차 엔트로피 오차'를 사용한다. 활성화 함수로는 이진 분류는 Sigmoid 함수를, 다중 분류는 Softmax 함수를 사용한다.

Q6. Negative Sampling을 구현해주세요.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss
import collections

In [3]:
class EmbeddingDot:
  def __init__(self, W):
    self.embed = Embedding(W)
    self.params = self.embed.params
    self.grads = self.embed.grads
    self.cache = None
  
  def forward(self, h, idx):
    target_W = self.embed.forward(idx)
    out = np.sum(target_W * h, axis = 1)

    self.cache = (h, target_W)
    return out

  def backward(self, dout):
    h, target_W = self.cache
    dout = dout.reshape(dout.shape[0], 1)

    dtarget_W = dout * h
    self.embed.backward(dtarget_W)
    dh = dout * target_W
    return dh

In [4]:
class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None
        
        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1
            
        vocab_size = len(counts)
        self.vocab_size = vocab_size
        
        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]
            
        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)
        
    def get_negative_sample(self, target):
        batch_size = target.shape[0]
        
        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)
            
            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0 
                p /= p.sum()  # 다시 정규화
                negative_sample[i, :] = np.random.choice(self.vocab_size,
                                                         size=self.sample_size,
                                                         replace=False, p=p)
                
        else:
            # GPU(cupy)로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, 
                                               size=(batch_size, self.sample_size), 
                                               replace=True, p=self.word_p)
            
        return negative_sample

In [5]:
class NegativeSamplingLoss:
  def __init__(self, W, corpus, power = 0.75, sample_size = 5):
      self.sample_size = sample_size
      self.sampler = UnigramSampler(corpus, power, sample_size)
      self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
      self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]
      self.params, self.grads = [], []
      for layer in self.embed_dot_layers:
        self.params += layer.params
        self.grads += layer.grads

  def forward(self, h, target):
      batch_size = target.shape[0]
      negative_sample = self.sampler.get_negative_sample(target)

      #긍정적 예 순전파
      score = self.embed_dot_layers[0].forward(h, target)
      correct_label = np.ones(batch_size, dtype = np.int32)
      loss = self.loss_layers[0].forward(score, correct_label)

      #부정적 예 순전파
      negative_label = np.zeros(batch_size, dtype = np.int32)
      for i in range(self.sample_size):
        negative_target = negative_sample[:, i]
        score = self.embed_dot_layers[1+i].forward(h, negative_target)
        loss += self.loss_layers[1 + i].forward(score, negative_label)

  def backward( self, dout=1): 
      dh = 0
      for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
          dscore = l0 .backward(dout) 
          dh += l1 .backward(dscore)

      return dh

Q7. 위 Embedding 계층과 Negative Sampling을 추가한 개선된 CBOW 모델을 Class형태로 구현해주세요.

In [6]:
# chap04/cbow.py
import sys
sys.path.append('..')
from common.np import *
from common.layers import Embedding
from negative_sampling_layer import NegativeSamplingLoss

In [7]:
# 아래에 코드를 작성해주세요.
class CBOW : 
    def __init__(self, vocab_size, hidden_size, window_size, corpus): 
        V, H = vocab_size, hidden_size 

        # 가중치 초기화 
        W_in = 0.01 * np.random.randn(V, H).astype('f') 
        W_out = 0.01 * np.random.randn(V, H).astype ('f') 

        # 계층 생성 
        self.in_layers = [ ] 
        for i in range (2 * window_size): 
            layer = Embedding(W_in)           # Embedding 계층 사용 
            self.in_layers.append(layer) 
        self.ns_loss = NegativeSamplingloss(W_out ,corpus, power = 0.75, sampe_size = 5)

        # 모든 가중치와 기울기를 배열에 모은다. 
        layers = self.in_layers + [self.ns_loss] 
        self.params, self.grads = [ ], [ ] 
        for layer in layers: 
            self.params += layer.params 
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다 . 
        self.word_vecs = W_in

    def forward (self, contexts, target): 
        h = 0 
        for i, layer in enumerate (self.in_layers): 
            h += layer.forward(contexts[:, i]) 
        h *= 1 / len(self.in_layers) 
        loss = self.ns_loss.forward(h, target) 
        return loss 
    def backward (self, dout=1): 
        dout = self.ns_loss.backward(dout) 
        dout *= 1 / len (self.in_layers)
        for layer in self.in_layers: 
            layer.backward(dout) 
        return None 
  

Q8. CBOW Model을 학습하여 파라미터를 pkl파일로 저장해주세요.

In [3]:
# chap04/train.py
import sys
sys.path.append('..')
import numpy as np
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
# config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb

In [4]:
# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)
  

In [10]:
# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

In [ ]:
# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

In [16]:
# pkl 파일 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

Q9. pkl파일을 불러와 예시 query를 작성해 결과를 보고, 유추(analogy)작업을 수행해주세요.

In [13]:
# pkl 파일 불러오기
import sys
sys.path.append('..')
from common.util import most_similar, analogy
import pickle

pkl_file = 'cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

In [14]:
# 예시 query 작성 후 비슷한 단어 살펴보기
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)


[query] you
 we: 0.72705078125
 i: 0.7080078125
 your: 0.6142578125
 someone: 0.59521484375
 they: 0.591796875

[query] year
 month: 0.86767578125
 week: 0.78564453125
 summer: 0.76416015625
 spring: 0.744140625
 decade: 0.6884765625

[query] car
 luxury: 0.62109375
 window: 0.60302734375
 cars: 0.595703125
 auto: 0.59033203125
 truck: 0.57763671875

[query] toyota
 coated: 0.62255859375
 honda: 0.5947265625
 chevrolet: 0.59423828125
 z: 0.58935546875
 ford: 0.58544921875


In [15]:
# 유추(analogy) 작업 수행
print('-'*50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)

--------------------------------------------------

[analogy] king:man = queen:?
 a.m: 5.328125
 woman: 4.86328125
 naczelnik: 4.8515625
 carolinas: 4.51171875
 wife: 4.3828125

[analogy] take:took = go:?
 went: 4.37109375
 came: 4.23828125
 're: 4.1953125
 were: 4.13671875
 eurodollars: 4.08984375

[analogy] car:cars = child:?
 a.m: 5.86328125
 rape: 5.75
 children: 5.0703125
 adults: 4.71484375
 incest: 4.6640625

[analogy] good:better = bad:?
 rather: 5.421875
 more: 5.33203125
 less: 5.20703125
 greater: 3.890625
 fewer: 3.748046875


Q10. 개선전 CBOW모델을 동일한 데이터셋으로 학습하여 나온 결과를 개선된 CBOW모델의 학습 결과와 비교해서 분석해주세요. (시간, 학습 결과 등)

In [1]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul, SoftmaxWithLoss


class SimpleCBOW:
    def __init__(self, vocab_size, hidden_size):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(H, V).astype('f')

        # 계층 생성
        self.in_layer0 = MatMul(W_in)
        self.in_layer1 = MatMul(W_in)
        self.out_layer = MatMul(W_out)
        self.loss_layer = SoftmaxWithLoss()

        # 모든 가중치와 기울기를 리스트에 모은다.
        layers = [self.in_layer0, self.in_layer1, self.out_layer]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h0 = self.in_layer0.forward(contexts[:, 0])
        h1 = self.in_layer1.forward(contexts[:, 1])
        h = (h0 + h1) * 0.5
        score = self.out_layer.forward(h)
        loss = self.loss_layer.forward(score, target)
        return loss

    def backward(self, dout=1):
        ds = self.loss_layer.backward(dout)
        da = self.out_layer.backward(ds)
        da *= 0.5
        self.in_layer1.backward(da)
        self.in_layer0.backward(da)
        return None


In [4]:
# coding: utf-8
import sys
sys.path.append('..')  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from common.trainer import Trainer
from common.optimizer import Adam
from simple_cbow import SimpleCBOW
from common.util import preprocess, create_contexts_target, convert_one_hot

window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

In [7]:
len(target)

929587

In [3]:

model = SimpleCBOW(vocab_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

word_vecs = model.word_vecs
for word_id, word in id_to_word.items():
    print(word, word_vecs[word_id])

92만개 데이터로 인하여 계산량 증가로 torch가 죽는 현상이 발생됨, 이로써 이전 CBOW보다 지금 CBOW가 성능면에서 좋다고 볼 수 있다.